# Simulating diffusion bridge for periodic sine drift with some current methods

In [ ]:
import DiffusionBridge as db
import math
import torch
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
# problem settings
d = 1
theta = torch.tensor(math.pi)
# alpha = torch.tensor(1.0)
alpha = torch.tensor(4.0)
f = lambda t,x: alpha * torch.sin(x - theta)  
sigma = torch.tensor(1.0)
T = torch.tensor(1.0)
M = 50
diffusion = db.diffusion.model(f, sigma, d, T, M)

# # initial and terminal constraints
X0 = 0.1 * torch.ones(d)
XT = 1.0 * torch.ones(d)

# repetitions
N = 2**10
R = 100

In [ ]:
# learn backward diffusion bridge process with score matching
epsilon = 1e-3
minibatch = 100
num_iterations = 500
learning_rate = 0.01
ema_momentum = 0.99
output = diffusion.learn_score_transition(X0, XT, epsilon, minibatch, num_iterations, learning_rate, ema_momentum)
score_transition_net = output['net']

# simulate backward diffusion bridge (BDB) process with approximate score
BDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, N, modify = False)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal
    
    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    BDB['ess'][r] = ess
    BDB['logestimate'][r] = log_transition_estimate
    
    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, 1, modify = False)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    BDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)), 
          ' Accept rate: ' + str(float(accept_rate)))

# simulate modified backward diffusion bridge (MBDB) process with approximate score
MBDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, N, modify = True)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    MBDB['ess'][r] = ess
    MBDB['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, 1, modify = True)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    MBDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))

In [ ]:
# use MBDB importance sampler with large number of sample paths as ground truth
N_large = 2**18
with torch.no_grad():
    output = diffusion.simulate_bridge_backwards(score_transition_net, X0, XT, epsilon, N_large, modify = True)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
log_target = diffusion.law_bridge(trajectories) 
log_weights = log_target - log_proposal
    
# importance sampling
max_log_weights = torch.max(log_weights)
weights = torch.exp(log_weights - max_log_weights)
log_transition_density = torch.log(torch.mean(weights)) + max_log_weights


In [ ]:
# learn forward diffusion bridge process with score matching
epsilon = 1e-3
minibatch = 100
num_iterations = 500
learning_rate = 0.01
ema_momentum = 0.99
output = diffusion.learn_score_marginal(score_transition_net, X0, XT, epsilon, minibatch, num_iterations, learning_rate, ema_momentum)
score_marginal_net = output['net']

# simulate forward diffusion bridge (FDB) process using approximate score
FDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, N, modify = False)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    FDB['ess'][r] = ess
    FDB['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, 1, modify = False)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    FDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))
          
# simulate modified forward diffusion bridge (MFDB) process using approximate score
MFDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}
for r in range(R):
    with torch.no_grad():
        output = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, N, modify = True)
        trajectories = output['trajectories']
        log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    MFDB['ess'][r] = ess
    MFDB['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_bridge_forwards(score_transition_net, score_marginal_net, X0, XT, epsilon, 1, modify = True)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    MFDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))
        

In [ ]:
# forward diffusion (FD) method of Pedersen (1995)
drift = f
FD = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}

for r in range(R):
    output = diffusion.simulate_proposal_bridge(drift, X0, XT, N)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)    
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    FD['ess'][r] = ess
    FD['logestimate'][r] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    FD['acceptrate'][r] = accept_rate
    
    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))


In [ ]:
# modified diffusion bridge (MDB) method of Durham and Gallant (2002)
drift = lambda t,x: (XT - x) / (T - t)
MDB = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}

for r in range(R):
    output = diffusion.simulate_proposal_bridge(drift, X0, XT, N, modify = True)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal
    
    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    MDB['ess'] = ess
    MDB['logestimate'] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1, modify = True)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    MDB['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))

In [ ]:
# diffusion bridge proposal of Clark (1990) and Delyon and Hu (2006) (CDH)
drift = lambda t,x: f(t,x) + (XT - x) / (T - t)
CDH = {'ess' : torch.zeros(R), 'logestimate' : torch.zeros(R), 'acceptrate' : torch.zeros(R)}

for r in range(R):
    output = diffusion.simulate_proposal_bridge(drift, X0, XT, N, modify = False)
    trajectories = output['trajectories']
    log_proposal = output['logdensity']
    log_target = diffusion.law_bridge(trajectories) 
    log_weights = log_target - log_proposal

    # importance sampling
    max_log_weights = torch.max(log_weights)
    weights = torch.exp(log_weights - max_log_weights)
    norm_weights = weights / torch.sum(weights)
    ess = 1.0 / torch.sum(norm_weights**2)
    log_transition_estimate = torch.log(torch.mean(weights)) + max_log_weights
    CDH['ess'] = ess
    CDH['logestimate'] = log_transition_estimate

    # independent Metropolis-Hastings
    initial = diffusion.simulate_proposal_bridge(drift, X0, XT, 1, modify = False)
    current_trajectory = initial['trajectories']
    current_log_proposal = initial['logdensity'] 
    current_log_target = diffusion.law_bridge(current_trajectory)
    current_log_weight = current_log_target - current_log_proposal
    num_accept = 0
    for n in range(N):
        proposed_trajectory = trajectories[n, :, :]
        proposed_log_weight = log_weights[n]
        log_accept_prob = proposed_log_weight - current_log_weight

        if (torch.log(torch.rand(1)) < log_accept_prob):
            current_trajectory = proposed_trajectory.clone()
            current_log_weight = proposed_log_weight.clone()  
            num_accept += 1
    accept_rate = num_accept / N
    CDH['acceptrate'][r] = accept_rate

    # print
    print('Repeat: ' + str(r) + 
          ' ESS%: ' + str(float(ess * 100 / N)) + 
          ' log-transition density: ' + str(float(log_transition_estimate)),
          ' Accept rate: ' + str(float(accept_rate)))

In [ ]:
# compare ESS
print('FD ESS%: ' + str(float(torch.mean(FD['ess']) * 100 / N)))
print('MDB ESS%: ' + str(float(torch.mean(MDB['ess']) * 100 / N)))
print('CDH ESS%: ' + str(float(torch.mean(CDH['ess']) * 100 / N)))
print('BDB ESS%: ' + str(float(torch.mean(BDB['ess']) * 100 / N)))
print('FDB ESS%: ' + str(float(torch.mean(FDB['ess']) * 100 / N)))
print('MBDB ESS%: ' + str(float(torch.mean(MBDB['ess']) * 100 / N)))
print('MFDB ESS%: ' + str(float(torch.mean(MFDB['ess']) * 100 / N)))


In [ ]:
# compare RMSE of log-transition density 
print('FD RMSE: ' + str(float(torch.sqrt(torch.mean((FD['logestimate'] - log_transition_density)**2)))))
print('MDB RMSE: ' + str(float(torch.sqrt(torch.mean((MDB['logestimate'] - log_transition_density)**2)))))
print('CDH RMSE: ' + str(float(torch.sqrt(torch.mean((CDH['logestimate'] - log_transition_density)**2)))))
print('BDB RMSE: ' + str(float(torch.sqrt(torch.mean((BDB['logestimate'] - log_transition_density)**2)))))
print('FDB RMSE: ' + str(float(torch.sqrt(torch.mean((FDB['logestimate'] - log_transition_density)**2)))))
print('MBDB RMSE: ' + str(float(torch.sqrt(torch.mean((MBDB['logestimate'] - log_transition_density)**2)))))
print('MFDB RMSE: ' + str(float(torch.sqrt(torch.mean((MFDB['logestimate'] - log_transition_density)**2)))))

In [ ]:
# compare indepedent Meteropolis-Hastings acceptance rate
print('FD acceptance%: ' + str(float(torch.mean(FD['acceptrate'] * 100))))
print('MDB acceptance%: ' + str(float(torch.mean(MDB['acceptrate'] * 100))))
print('CDH acceptance%: ' + str(float(torch.mean(CDH['acceptrate'] * 100))))
print('BDB acceptance%: ' + str(float(torch.mean(BDB['acceptrate'] * 100))))
print('FDB acceptance%: ' + str(float(torch.mean(FDB['acceptrate'] * 100))))
print('MBDB acceptance%: ' + str(float(torch.mean(MBDB['acceptrate'] * 100))))
print('MFDB acceptance%: ' + str(float(torch.mean(MFDB['acceptrate'] * 100))))
